In [1]:
import pandas as pd
import os
from pandas import ExcelFile
from fuzzywuzzy import fuzz
from selenium.common.exceptions import *


def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

In [2]:
min_law_df = pd.read_csv('min_prof_aals.csv')
min_law_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [38]:
main_df = pd.read_csv(create_path('school_data', 'full_data_new_with_gender.csv'))
main_df.drop(columns = ['Unnamed: 0'], inplace=True)
main_df = main_df[main_df.duplicated() == False]
main_df.reset_index(inplace = True, drop = True)
dup_names = zip(main_df[main_df.duplicated(subset = ['First Name', 'Last Name']) == True]['First Name'], main_df[main_df.duplicated(subset = ['First Name', 'Last Name']) == True]['Last Name'])
dup_index_list = []
for first, last in dup_names:
    dup_index_list = dup_index_list + list(main_df[(main_df['First Name'] == first) & (main_df['Last Name'] == last)].index.values.astype(int))
dup_index = [0]*len(main_df)
for i in dup_index_list:
    dup_index[i] = 1
main_df['Duplicate'] = dup_index

,First Name,Last Name,Title,School,E-mail,Phone,Gender,Gender Label,Duplicate
46,David,Schwartz,Foley & Lardner Bascom Professor of Law,University of Wisconsin Law School,dsschwartz@wisc.edu,(608) 262-8150,male,1,1
49,Brad,Snyder,Ass't Prof.,University of Wisconsin Law School,bsnyder2@wisc.edu,(608) 262-1015,male,1,1
88,David,Schwartz,Professor of Law,Northwestern University Pritzker School of Law,david.schwartz@law.northwestern.edu,(312) 503-2646,male,1,1
107,David,Hyman,"Richard W. and Marie L. Corman Professor, Dire...",University of Illinois College of Law,dhyman@illinois.edu,(217) 333-0061,male,1,1
128,Melissa,Wasserman,"Professor, Richard and Anne Stockton Faculty S...",University of Illinois College of Law,mfwasser@illinois.edu,(217) 244-3960,female,0,1
339,Reuven,Avi-Yonah,Professor of Law,"University of California, Irvine School of Law",raviyonah@law.uci.edu,(949) 824-0066,male,1,1
360,Alexandra,Natapoff,Professor of Law,"University of California, Irvine School of Law",anatapoff@law.uci.edu,(949) 824-0066,female,0,1
688,Melissa,Wasserman,Charles Tilford McCormick Professor of Law,The University of Texas School of Law,mwasserman@law.utexas.edu,(512) 471-8079,female,0,1
704,Mark,Janis,William F. Starr Prof.,University of Connecticut School of Law,mark.janis@uconn.edu,(860) 570-5265,male,1,1
716,Richard,Parker,Professor of Law,University of Connecticut School of Law,richard.parker@uconn.edu,(202) 258-2617,male,1,1


In [39]:
names = [x[0] + ' ' + x[1] for x in zip(main_df['First Name'], main_df['Last Name'])]
schools = main_df['School']

In [40]:
min_names = [x[0] + ' ' + x[1] for x in zip(min_law_df['First Name'], min_law_df['Last Name'])]
min_schools =  min_law_df['School']
min_aals_names = list(min_law_df['AALS Name'])
min_aals_schools = list(min_law_df['AALS School'])
fuzz = list(min_law_df['Fuzz List'])

In [41]:
def check_google(school):
    short_url_list = []
    if school == 'Southwestern  university':
        school == 'Southwestern Law School'
    elif school == 'Brooklyn  university':
        school = 'Brooklyn Law School'
    search_name(school)
    elems = g_driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        try:
            url = elem.get_attribute("href")
        except StaleElementReferenceException:
            continue
        if '.edu' in url:
            if '.law.' in url:
                short_url = url.split('.law.')[1].split('.')[0]
            else:
                short_url = url.split('.')[1].split('.')[0]
            short_url_list.append(short_url)
#             print('.edu' + ' in: ' + short_url)
    return short_url_list

def search_name(name):
    g_driver.get("http://bing.com")
    search = g_driver.find_element_by_xpath('//*[@id="sb_form_q"]')
    search.send_keys(name)
    search.send_keys(Keys.RETURN)
    
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [42]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

g_driver = webdriver.Chrome()

In [47]:
labels = [0]*len(main_df)
min_name_list = ['NA']*len(main_df)
min_school_list = ['NA']*len(main_df)
fuzz_list = [0]*len(main_df)
for i in range(len(names)):
    if names[i] in min_aals_names:
        print(names[i])
        name_index = min_aals_names.index(names[i])
        if fuzz[name_index] > 66:
            school_1 = min_aals_schools[name_index] + ' university'
            school_2 = schools[i]
            school_list_1 = check_google(school_1)
            school_list_2 = check_google(school_2)
            min_name_list[i] = min_names[name_index]
            min_school_list[i] = min_aals_schools[name_index]
            fuzz_list[i] = fuzz[name_index]
            if intersection(school_list_1, school_list_2):
                labels[i] = 1

Marina Lao
Margaret Lewis
Richard Monette
Joyce Hughes
Michael Kang
Jide Nzelibe
Vikram Amar
Francis Boyle
Jay Kesan
Jamelle Sharpe
Suja Thomas
Melissa Wasserman
Sonia Katyal
Victoria Plaut
Sarah Song
Karen Tani
David Cruz
Daria Roithmayr
Alexia Brunet Marks
Paul Campos
Lakshman Guruswamy
Peter Huang
Jamal Greene
Olatunde Johnson
Alejandro Camacho
Jonathan Glater
Michele Goodwin
Kaaryn Gustafson
Annie Lai
David Law
L. Song Richardson
Shauhin Talesh
Karen Brown
W. Carter
Robert Cottrol
Theresa Gabaldon
Susan Jones
Cynthia Lee
Spencer Overton
Kim Forde-Mazrui
Andrew Hayashi
Daniel Ortiz
Mildred Robinson
A. Benjamin Spencer
Jose Alvarez
Harvey Dale
Kevin Davis
Peggy Davis
Melissa Murray
Kenji Yoshino
Richard Albert
Melissa Wasserman
Patrick Woolley
Sara Bronin
James Kwak
Willajeanne McLean
Sachin Pandya
William Araiza
Bennett Capers
Steven Dean
Samuel Murumba
Michael Kelly
Jean Ramirez
Mary Jo Wiggins
Shi-Ling Hsu
Frank Rudy Cooper
Elbert Robertson
Ragini Shah
Patrick Shin
Meredith Duncan

In [48]:
main_df['Minority Status'] = labels
main_df['Min List Name'] = min_name_list
main_df['Min School List'] = min_school_list
main_df

,First Name,Last Name,Title,School,E-mail,Phone,Gender,Gender Label,Duplicate,Minority Status,Min List Name,Min School List
0,Michael,Ambrosio,Professor of Law,Seton Hall University School of Law,michael.ambrosio@shu.edu,(973) 642-8807,male,1,0,0,NA,NA
1,Gaia,Bernstein,"Michael J. Zimmer Professor of Law, Director o...",Seton Hall University School of Law,gaia.bernstein@shu.edu,(973) 642-8494,female,0,0,0,NA,NA
2,Kristen,Boon,Associate Dean for Faculty Research and Develo...,Seton Hall University School of Law,kristen.boon@shu.edu,(973) 642-8958,female,0,0,0,NA,NA
3,Kathleen,Boozang,Law School Dean and Professor of law,Seton Hall University School of Law,kathleen.boozang@shu.edu,(973) 642-8501,female,0,0,0,NA,NA
4,Angela,Carmella,Professor of Law,Seton Hall University School of Law,angela.carmella@shu.edu,(973) 642-8843,female,0,0,0,NA,NA
5,Carl,Coleman,"Professor of Law, and Academic Director of Div...",Seton Hall University School of Law,carl.coleman@shu.edu,(201) 204-9512,male,1,0,0,NA,NA
6,Jenny-Brooke,Condon,Associate Professor of Law,Seton Hall University School of Law,jenny-brooke.condon@shu.edu,(973) 642-8307,unknown,2,0,0,NA,NA
7,John,Cornwell,Professor of Law,Seton Hall University School of Law,kip.cornwell@shu.edu,(973) 642-8498,male,1,0,0,NA,NA
8,Mark,Denbeaux,Professor of Law,Seton Hall University School of Law,mark.denbeaux@shu.edu,(973) 642-8822,male,1,0,0,NA,NA
9,Linda,Fisher,Professor of Law,Seton Hall University School of Law,linda.fisher@shu.edu,(973) 642-8700,female,0,0,0,NA,NA


In [49]:
main_df.to_csv('law_prof_data_gender_min_de_dup.csv')

In [33]:
def closest_three(min_name, min_school, names_schools):
    max_fuzz = [0, 0, 0]
    name_list = ['', '', '']
    school_list = ['', '', '']
    for name, school in names_schools:
        fuzz_ratio = fuzz.ratio(min_name, name)
        if fuzz_ratio > max_fuzz[2]:
            name_list.pop(2)
            school_list.pop(2)
            max_fuzz.pop(2)
            name_list.insert(2, name)
            max_fuzz.insert(2, fuzz_ratio)
            school_list.insert(2, school)
            max_fuzz, name_list, school_list = (list(t) for t in zip(*sorted(zip(max_fuzz, name_list, school_list), reverse = True)))
    return list(zip(max_fuzz, name_list, school_list))

In [40]:
master_list = []
for min_name, min_school in min_names_schools:
    closest_list = closest_three(min_name, min_school, names_schools)
    master_list.append(closest_list)

In [67]:
fuzz_1 = [master_list[i][0][0] for i in range(len(master_list))]
fuzz_2 = [master_list[i][1][0] for i in range(len(master_list))]
fuzz_3 = [master_list[i][2][0] for i in range(len(master_list))]
name_1 = [master_list[i][0][1] for i in range(len(master_list))]
name_2 = [master_list[i][1][1] for i in range(len(master_list))]
name_3 = [master_list[i][2][1] for i in range(len(master_list))]
school_1 = [master_list[i][0][2] for i in range(len(master_list))]
school_2 = [master_list[i][1][2] for i in range(len(master_list))]
school_3 = [master_list[i][2][2] for i in range(len(master_list))]
near_df = pd.DataFrame({'Fuzz 1': fuzz_1, 'Fuzz 2': fuzz_2, 'Fuzz 3': fuzz_3, 
              'Name 1': name_1, 'Name 2': name_2, 'Name 3': name_3, 
             'School 1': school_1, 'School 2': school_2, 'School 3': school_3})
df = pd.concat([min_law_df, near_df], axis = 1,  sort = False)
df.to_csv('min_law_nearest_names.csv')

In [138]:
elems = g_driver.find_elements_by_xpath("//a[@href]")

In [139]:
for elem in elems:
#     print(elem.get_attribute("href"))
    if '.edu' in elem.get_attribute("href"):
        url = elem.get_attribute("href")
        print(elem.get_attribute("href"))
        short_url = url.split('.')[1].split('.')[0]
        print(short_url)

http://www.smu.edu/
smu
https://www.smu.edu/
smu
https://www.smu.edu/admission/
smu
http://www.smu.edu/
smu


In [196]:
removed_values = [output[i] for i in index_list]

In [185]:
min_profs = [output[i] for i in range(len(output)) if i not in index_list]

In [198]:
def create_dataframe(prof_list):
    min_list_name = [x[0][0] for x in prof_list]
    min_list_school = [x[0][1] for x in prof_list]
    data_list_name = [x[1][0] for x in prof_list]
    data_list_school = [x[1][1] for x in prof_list]
    fuzz_ratio = [x[2] for x in prof_list]
    min_name_df = pd.DataFrame({'Min List Name': min_list_name, 'Data List Name': data_list_name,'Min List School': min_list_school, 'Data List School': data_list_school, 'Fuzz Ratio': fuzz_ratio})
    return min_name_df

In [200]:
incorrect_name_df = create_dataframe(removed_values)
correct_name_df = create_dataframe(min_profs)
incorrect_name_df.to_csv('incorrect_names.csv')
correct_name_df.to_csv('correct_names.csv')

In [206]:
min_index_list_df = [names.index(x) for x in list(correct_name_df['Data List Name'])]

In [213]:
min_var = [1 if i in min_index_list_df else 0 for i in range(len(main_df))]

In [214]:
main_df['Minority'] = min_var

In [216]:
main_df.to_csv('law_prof_data_gender_min.csv')